In [12]:
# Assert config + dataset_external table

from google.cloud import bigquery
from google.api_core.exceptions import NotFound

PROJECT_ID  = "infinite-mantra-480821-v7"
DATASET_ID  = "telco_churn_ds"
EXT_TABLE   = "customers"          # external (points to GCS)
NATIVE_TBL  = "customers_native"   # native (we materialize here)

bq = bigquery.Client(project=PROJECT_ID)

full_ext = f"{PROJECT_ID}.{DATASET_ID}.{EXT_TABLE}"
full_nat = f"{PROJECT_ID}.{DATASET_ID}.{NATIVE_TBL}"

print("Client project:", bq.project)
print("External:", full_ext)
print("Native  :", full_nat)

# Dataset location must be US
ds = bq.get_dataset(f"{PROJECT_ID}.{DATASET_ID}")
print("Dataset location:", ds.location)

# External table must exist (and points to your CSV)
ext = bq.get_table(full_ext)
print("External table exists ")
print("External source:", ext.external_data_configuration.source_uris)

Client project: infinite-mantra-480821-v7
External: infinite-mantra-480821-v7.telco_churn_ds.customers
Native  : infinite-mantra-480821-v7.telco_churn_ds.customers_native
Dataset location: US
External table exists 
External source: ['gs://mlops-telco-bigdata/Telco-Customer-Churn.csv']


In [13]:
#Reading from external table
bq.query(f"SELECT COUNT(*) AS n FROM `{full_ext}`").to_dataframe()

,n
0,7043


In [14]:
#Create/replace native table

from google.api_core.exceptions import BadRequest, GoogleAPICallError

sql_ingest = f"""
CREATE OR REPLACE TABLE `{full_nat}` AS
SELECT
  customerID,
  gender,
  SeniorCitizen,
  Partner,
  Dependents,
  SAFE_CAST(tenure AS INT64) AS tenure,  -- works if tenure is already INT64 or STRING
  PhoneService,
  MultipleLines,
  InternetService,
  OnlineSecurity,
  OnlineBackup,
  DeviceProtection,
  TechSupport,
  StreamingTV,
  StreamingMovies,
  Contract,
  PaperlessBilling,
  PaymentMethod,
  SAFE_CAST(NULLIF(TRIM(CAST(MonthlyCharges AS STRING)), '') AS FLOAT64) AS MonthlyCharges,
  SAFE_CAST(NULLIF(TRIM(CAST(TotalCharges   AS STRING)), '') AS FLOAT64) AS TotalCharges,
  Churn
FROM `{full_ext}`;
"""

try:
    job = bq.query(sql_ingest)
    job.result()  # wait for completion
    print(" Ingestion job done | job_id:", job.job_id, "| state:", job.state, "| errors:", job.errors)
except BadRequest as e:
    print(" BigQuery BadRequest")
    print("Message:", getattr(e, "message", str(e)))
    if getattr(e, "errors", None):
        for err in e.errors:
            print("-", err.get("message"), "| location:", err.get("location"), "| reason:", err.get("reason"))
    raise
except GoogleAPICallError as e:
    print(" Google API error:", getattr(e, "message", str(e)))
    raise


 Ingestion job done | job_id: 37a1c074-5d40-41f3-bea3-f9ca725bab17 | state: DONE | errors: None


In [15]:
# Confirming Native table exist and Rows
from google.api_core.exceptions import NotFound

try:
    tbl = bq.get_table(full_nat)
    print(" Native table exists:", tbl.full_table_id, "| rows:", tbl.num_rows)
except NotFound:
    print(" Native table NOT found:", full_nat)

# Count & preview
bq.query(f"SELECT COUNT(*) AS n FROM `{full_nat}`").to_dataframe()
bq.query(f"SELECT * FROM `{full_nat}` LIMIT 5").to_dataframe()


 Native table exists: infinite-mantra-480821-v7:telco_churn_ds.customers_native | rows: 7043


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,9426-SXNHE,Female,0,False,False,2,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Bank transfer (automatic),18.75,53.15,False
1,3387-PLKUI,Female,0,True,True,13,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,18.80,251.25,False
2,3806-YAZOV,Female,0,False,False,3,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,18.80,56.00,False
3,0620-XEFWH,Male,0,True,True,4,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,18.85,84.20,False
4,8992-CEUEN,Female,0,False,False,1,True,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Electronic check,18.85,18.85,False
